<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/math-and-architectures-of-deep-learning/introduction-to-vectors-matrices-and-tensors/05_spectral_decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Spectral Decomposition

Let's consider square symmetric matrix with $n$ rows and $n$ columns.

And also assuming it is non-singular (i.e., its determinant is non-zero) and
it has $n$ distinct eigenvalues, it will have $n$ mutually orthogonal
eigenvectors (i.e., their dot-products are zero and geometrically
they represent perpendicular vectors).

Let the eigenvalues be $\lambda_{1}, \lambda_{2}, \cdots, \lambda_{n}$
 and the eigenvectors be $\vec{e}_{1}, \vec{e}_{2}, \cdots, \vec{e}_{n}$.

This matrix can be decomposed as,

 $A_{n, n} = S \Sigma S^{T}$.

where,

$$
S = 
\begin{bmatrix}
        e_1 & e_2 & ... & e_n \\
\end{bmatrix}
$$

Thus,

$$
A = 
\begin{bmatrix}
        e_1 & e_2 & ... & e_n \\
\end{bmatrix}
\begin{bmatrix}
        \lambda_1 & 0 & ... & 0 \\
        0 & \lambda_2 & ... & 0 \\
        ... & ... & ... \\
        ... & ... & ... \\
        0 & 0 & ... & \lambda_n \\
\end{bmatrix}
\begin{bmatrix}
        e_1^T \\
        e_2^T \\
        ... \\
        ... \\
        e_n^T \\
\end{bmatrix}
$$

The above equation can be rewritten as equivalently, 

$A_{n, n} = \lambda_{1} \vec{e}_{1} \vec{e}_{1}^{T} + \lambda_{2} \vec{e}_{2} \vec{e}_{2}^{T}+ ... + \lambda_{n} \vec{e}_{n} \vec{e}_{n}^{T}$.


**This is the spectral decomposition of a symmetric matrix.**

##Setup

In [ ]:
import torch
import torch.linalg as LA

import numpy as np
import math
from math import cos, sin, radians
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

In [ ]:
torch.manual_seed(42)

##Spectral decomposition of symmetric matrix

Let us reconsider the spectral decomposition of a symmetric matrix.

In [ ]:
def spectral_decomposition(A):
  """
  Returns the spectral decomposition of a square symmetric matrix
  """
  # Assert input matrix is square and symmetric.
  # Check if number of rows and columns match and
  # if the matrix and its transpose are identical.
  assert len(A.shape) == 2 and A.shape[0] == A.shape[1] and torch.all(A == A.T)

  # calculate the eigenvectors and values
  l, e = torch.linalg.eig(A)

  # the numpy unique function checks if all elements of an array are distinct.
  assert len(torch.unique(l.real)) == A.shape[0], "Eigen values are not distinct!"

  # Let us define a tensor of shape n * n * n to hold the individual terms of the spectral decomposition.
  # This tensor can be thought of as a collection of n matrices. The ith matrix is the ith term of the decomp - lambda_i * e_i * e_i^T.
  # Numpy function zeros creates an array filled with zeros.
  components = torch.zeros((A.shape[0], A.shape[0], A.shape[0]))

  for i, lambda_i in enumerate(l):
    e_i = e[:, i]
    # We use reshape to force the vector to be a row vector. 
    e_i = e_i.reshape((3, 1))
    # We add the the corresponding value to the components
    components[i, :, :] = (lambda_i * torch.matmul(e_i, e_i.T)).real
  return components

In [ ]:
A = torch.tensor([
  [1, 2, 1],
  [2, 2, 3],
  [1, 3, 3]
]).float()

components = spectral_decomposition(A)

# We createa new matrix A1 as sum of the components.
# axis=0 specifies that we should be summing along axis 0
A1 = components.sum(axis=0)

# Then assert A1 is the same as the orginal matrix A.
# Success of this assert verifies the math.
assert torch.allclose(A, A1)

print(f"A: \n{A}")
print(f"A1: \n{A1}")

A: 
tensor([[1., 2., 1.],
        [2., 2., 3.],
        [1., 3., 3.]])
A1: 
tensor([[1.0000, 2.0000, 1.0000],
        [2.0000, 2.0000, 3.0000],
        [1.0000, 3.0000, 3.0000]])
